In [1]:
import os, gensim, re
import numpy as np
class MySentences(object):
    def __init__(self, dirname):
        self.dirname = dirname
 
    def __iter__(self):
        for fname in os.listdir(self.dirname):
            with open(self.dirname + '/' + fname, 'r') as f:
                context = f.read()
            sentences = self.clean_context(context)[40 : -3]
            for s in sentences:
                yield s
            
    def clean_context(self, context):
        words = re.sub('[\r]|[\n]', ' ', context)
        words = re.sub('([a-z0-9])(\W{2,})', r'\1 \2', words)
        words = re.sub('(\W{2,})([a-zA-z0-9])', r'\1 \2', words)
        words = re.sub('\s+', ' ', words)
        words = re.sub('\W{3,}', '\n', words)
        raw = words.split('\n')
        sentences = []
        for r in raw:
            s = r.split(' ')
            if len(s) > 3:
                sentences.append(s)
        return sentences
    def get_sentences(self):
        for fname in os.listdir(self.dirname):
            with open(self.dirname + '/' + fname, 'r') as f:
                context = f.read()
            sentences = self.clean_context(context)[40 : -3]
            for s in sentences:
                if len(s) > 5:
                    yield s
redundantWord = ['the', 'a']
                    
def cloze_test(string, index, candidates):
    temp = string[:]
#     print temp[index]
    del temp[index]
    nc = len(candidates)
    nw = len(temp)
    arr = np.zeros((nc, nw))
#     weight = np.zeros(nw)
    for j in xrange(nw):
#         weight[j] = index - abs(index - j)
        for i in xrange(nc):
            if len(candidates[i]) <= 4:
                continue
            if temp[j] not in redundantWord:
                if temp[j] in model.vocab and candidates[i] in model.vocab:
                    if temp[j] != candidates[i]:
                        arr[i, j] = model.similarity(candidates[i], temp[j])
        l, r = arr[:, j].min(), arr[:, j].max()
        if r > l:
            arr[:, j] = (arr[:, j] - l) / (r - l)
    return arr
#     weight[j] /= float(index)
#     return np.dot(arr, weight)

In [279]:
sentences = MySentences('harry_potter')
model = gensim.models.Word2Vec(sentences, window = 3, min_count = 10, size = 100, iter = 100)

In [350]:
model.most_similar(positive = ['Malfoy', 'Dumbledore'], negative = ['Harry'], topn = 10)

[('Voldemort', 0.5814064741134644),
 ('Quirrell', 0.5672532320022583),
 ('McGonagall', 0.5618100166320801),
 ('Snape', 0.558242678642273),
 ('Slughorn', 0.5175069570541382),
 ('Trelawney', 0.5160765647888184),
 ('Moody', 0.5088180899620056),
 ('You-Know-Who', 0.4961373805999756),
 ('Fudge', 0.4755627512931824),
 ('Sprout', 0.47143012285232544)]

In [349]:
model.similarity('Harry', 'Peter')

0.23077564783587678

In [214]:
sentences = MySentences('harry_potter')
model.train(sentences)

In [234]:
model.most_similar(positive = ['Harry'], topn = 10)

[('he', 0.6603592038154602),
 ('He', 0.5858224034309387),
 ('Hermione', 0.4437968134880066),
 ('she', 0.4352525472640991),
 ('Frank', 0.38196608424186707),
 ('him', 0.3726317882537842),
 ('Ron', 0.36592867970466614),
 ('they', 0.3517574667930603),
 ('Hagrid', 0.34963470697402954),
 ('Dumbledore', 0.3127173185348511)]

In [268]:
# 4846 vocabs
print len(model.vocab)
print list(model.vocab)[:1]

4846
['Aylar']


In [376]:
model.similarity('man', 'boy')

0.65237136780624727

In [11]:
# test = 'though could not remember the name'.split(' ')
model.most_similar('Malfoy')

[('Harry', 0.6313201785087585),
 ('he', 0.5895937085151672),
 ('Neville', 0.5751792192459106),
 ('Cho', 0.5731143355369568),
 ('him', 0.5708431601524353),
 ('Snape', 0.5683074593544006),
 ('Hermione', 0.5656015872955322),
 ('Ginny', 0.5607053637504578),
 ('Tci', 0.5568682551383972),
 ('Hagrid', 0.5517871379852295)]

In [9]:
# model.save('w2v_harry_potter')

In [2]:
model = gensim.models.Word2Vec.load('w2v_harry_potter')

In [11]:
# model.index2word[:10]  # 100
# v = gensim.models.word2vec.Vocab(a = 5, b = 3, c = 'asf', b_bb = '10')
# v.__str__()
model.vocab['I'].count

6381

In [270]:
# 43546 lines
sentences = MySentences('harry_potter').get_sentences()
with open('sentences.txt', 'w') as f:
    for s in sentences:
        if len(s) <= 10:
            for w in s:
                f.write(w+',')
            f.write('\n')
    

In [76]:
import random

def make_cloze(num):
    idx = range(0, 43546)
    random.shuffle(idx)
    idx = sorted(set(idx[: num]))
    
    curr_idx = 0
    sentences = []
    with open('sentences.txt', 'r') as f:
        for i in idx:
            for j in xrange(i - curr_idx): f.next()
            sentences.append(f.next().split(',')[:-1])
            curr_idx = i + 1
    ;
    options = []
    keys = model.vocab.keys()
    for s in sentences:
        l = len(s)
        idx = range(0, 4846)
        random.shuffle(idx)
        
        candidate = [keys[i] for i in idx[:3]]
        count = 0
        while count < 10:
            i = random.randint(0, l - 1)
            if len(s[i]) > 4 or count == 9:
                options.append((i, [s[i]] + candidate))
                break
            count += 1
        
                
    return sentences, options


def cloze_test2(string, index, candidates):
    temp = string[:]
    del temp[index]
    ref_vec = np.zeros(100)
    l = 0
    for w in temp:
        if w in model.vocab:
            ref_vec += model[w]
            l += 1
    ref_vec /= float(l)
    ref_vec = ref_vec / np.linalg.norm(ref_vec)
    nc = len(candidates)
    arr = np.zeros(nc)
    for i in xrange(nc):
        if len(candidates[i]) <= 4 or candidates[i] in temp:
            continue
        if candidates[i] in model.vocab:
            cand_vec = model[candidates[i]]
            arr[i] = (ref_vec * (cand_vec / np.linalg.norm(cand_vec))).sum()
#     l, r = arr[:, j].min(), arr[:, j].max()
#     if r > l:
#         arr[:, j] = (arr[:, j] - l) / (r - l)
    return arr


def cal_score(sentences, options):
    n = len(sentences)
    s = 0.
    ref = 0.
    for i in xrange(n):
#         print sentences[i]
#         print options[i][0], options[i][1]
        b = cloze_test2(sentences[i], options[i][0], options[i][1])
        if b.argmax() == 0: s += 1.
        if random.randint(0, 3) == 0: ref += 1.
    return s/float(n), ref/float(n)

for i in xrange(10):
    sentences, options = make_cloze(1000)
    score, ref = cal_score(sentences, options)
    print score, ',', ref

0.389 , 0.296
0.403 , 0.273
0.393 , 0.231
0.378 , 0.275
0.405 , 0.254
0.386 , 0.249
0.406 , 0.252
0.398 , 0.244
0.362 , 0.262
0.422 , 0.255


In [65]:
# a =['until','the','Cup','is'] # World
a = ['boy','is','to','me'] # nothing
# a = ['man','could','to','snakes'] # talk
# a = ['Frank',"didn't",'what','was'] # understand
b = [model[x] / np.linalg.norm(model[x]) for x in a]
c = np.asarray(b).mean(axis = 0)
model.most_similar([c])

[('me', 0.6290419101715088),
 ('you', 0.580627977848053),
 ('boy', 0.547135591506958),
 ('is', 0.5445059537887573),
 ('to', 0.5434709787368774),
 ('us', 0.5378933548927307),
 ('ll', 0.47368937730789185),
 ('t', 0.4723420739173889),
 ('You', 0.44806158542633057),
 ('I', 0.4452311396598816)]

In [6]:
arr = np.array([2.22,3.33,0,8.9,9.0])
arr.argmax()

4

In [61]:
arr = np.ones((2,3))
# arr_norm = arr / 
arr /  (np.dot(np.linalg.norm(arr, axis = 1), np.ones(arr.shape)))
# np.ones(arr.shape)

array([[ 0.28867513,  0.28867513,  0.28867513],
       [ 0.28867513,  0.28867513,  0.28867513]])

In [ ]:
arr = np.asarray([1,2,3,4])
(arr * arr).sum()